In [26]:
import pandas as pd
import numpy as np
import promptlayer

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chat_models import PromptLayerChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
WOLFRAM_ALPHA_APPID = os.getenv('WOLFRAM_ALPHA_APPID')
PROMPTLAYER_API_KEY = os.getenv('PROMPTLAYER_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')



from memory import PineconeMemory


# Initialize memory and make sure it is empty.
# this is particularly important for indexing and referencing pinecone memory

memory = PineconeMemory()
memory.clear()
full_message_history = []

print('Using memory of type: ' + memory.__class__.__name__)


# Initialize the agent
openai = promptlayer.openai
chat = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)


In [48]:
reviews = pd.read_csv('reviews.csv')
querry = {0: "What are the top traits that customers appreciate the most about a product? Provide a description of each trait. ",
          1: "What are the top traits that customers appreciate the least about a product? Provide a description of each trait. ",
          2: "What are the improvements that can be brought to this product? Provide a description of each trait. "}

SystemMessage = """Answer the question based on the context and reviews below. You will answer with bulletpoints and extra clarity as a profesional developer. If the question cannot be answered using the information provided answer with "I don't know".

Context: You are looking at a product sold on amazon.com. We are a competing product development team. Our scope is to better understand the clients need with our product in order to improve the product. """

system_message_prompt = SystemMessagePromptTemplate.from_template(SystemMessage)


HumanMessage = """Question:  {inputQuestion}
                Reviews: {inputReviews} """

human_message_prompt = HumanMessagePromptTemplate.from_template(HumanMessage)

AiMessage = """Answer: """
ai_message_prompt = AIMessagePromptTemplate.from_template(AiMessage)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, ai_message_prompt])

In [ ]:
# Pass the reviews and querry to the promptlayer, get the results and add them to the memory

if len(querry) > 0:
    for i in range(len(reviews)):
        for j in range(len(querry)):
            # PROMPTLAYER
            prompt = chat_prompt.format_prompt(
                inputQuestion=querry[j],
                inputReviews=reviews['review'][i]
            ).to_messages()
            results = chat(prompt) 
            print(results)
       
            full_memory_to_add = f" Human Message: {querry[j]} " \
                f"\n Assistant Reply: {results} " \
                f"\n Reviews: {reviews['review'][i]} "
            memory.add(full_memory_to_add)

            short_memory_to_add = f" Human Question: {querry[j]} " \
                f"\n Assistant Answer: {results} "
            memory.add(short_memory_to_add)

# template_dict = promptlayer.prompts.get("reviewsPromptFeatures_test1")
# https://python.langchain.com/en/latest/modules/models/chat/integrations/promptlayer_chatopenai.html


# memory.get_relevant("Assistant Reply", num_relevant=6)


memory.get_stats()

In [49]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import json

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=SERPAPI_API_KEY)

In [119]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [123]:
response = agent({"input":"Prepare a prompt for a LLM model to generate a description of a product. "
                    "This is the AISN of the product: B08TVXQ5S1. https://h-amazon-data-scraper2.p.rapidapi.com/products/B08TVXQ5S1/reviews "})



> Entering new AgentExecutor chain...
 I need to find out what the product is and what it does.
Action: Search
Action Input: B08TVXQ5S1
Observation: Our indoor hydroponics growing system, Simply fill water, no need soil, a sleek, indoor garden that allows you to grow up to 12 different herbs, veggies, or ...
Thought: I need to find out what people think about the product.
Action: Search
Action Input: B08TVXQ5S1 reviews
Observation: ASIN, ‎B08TVXQ5S1. Customer Reviews, 4.2 out of 5 stars 1,526Reviews. Best Sellers Rank. #6,808 in Patio, Lawn & Garden (See Top 100 in Patio, Lawn & Garden).
Thought: I now know the product and what people think about it.
Final Answer: The Simply Hydroponics Growing System (ASIN: B08TVXQ5S1) is a sleek, indoor garden that allows you to grow up to 12 different herbs, veggies, or plants. It has an average customer rating of 4.2 out of 5 stars from 1,526 reviews and is ranked #6,808 in Patio, Lawn & Garden.

> Finished chain.


I need a function that will receive a series of prompts that are passed to the GPT, it will save them to memory toghether with the reply the GPT gets to each of those prompts


My code is:
